# Минимальные остовы

Рассмотрим такую задачу: дан неориентированный граф с рёбрами положительной стоимости, и мы в нем хотим выбрать такой подграф, который будет. Такой подграф называется минимальным остовным деревом (англ. minimum spanning tree).

Остов — каркас, скелет. Ударение на первый слог, хотя так мало кто произносит.

**Остовное дерево является деревом**. Если бы оно не было деревом, то где-то был бы цикл, из которого можно было бы что-то удалить и получить граф поменьше.

**Лемма о безопасном ребре.**

Пусть есть MST, в котором минимального ребра нет. Тогда если мы добавим это ребро, то образуется цикл, из которого можно выкинуть любое другое, получив остов меньшего веса.

# Алгоритм Прима

Это мало когда применяется, но, так же как и с Дейкстрой, он может и работать без лагорифма, когда граф полный. Нужно просто забыть про приоритетную очередь и каждый раз за линию искать новую вершину.

Совсем наивная реализация за $O(nm)$ — каждый раз перебираем все рёбра:

In [ ]:
const int maxn = 1e5, inf = 1e9;
vector<int> from, to, weight;
bool used[maxn]

// считать все рёбра в массивы

used[0] = 1;
for (int i = 0; i < n-1; i++) {
    int opt_w = inf, opt_from, opt_to;
    for (int j = 0; j < m; j++)
        if (opt_w > weight[j] && used[from[j]] && !used[to[j]])
            opt_w = weight[j], opt_from = from[j], opt_to = to[j]
    used[opt_to] = 1;
    cout << opt_from << " " << opt_to << endl;
}

Реализация за $O(n^2)$:

In [ ]:
const int maxn = 1e5, inf = 1e9;
bool used[maxn];
vector< pair<int, int> > g[maxn];
int min_edge[maxn] = {inf}, best_edge[maxn];
min_edge[0] = 0;

// ...

for (int i = 0; i < n; i++) {
    int v = -1;
    for (int u = 0; u < n; j++)
        if (!used[u] && (v == -1 || min_edge[u] < min_edge[v]))
            v = u;
 
    used[v] = 1;
    if (v != 0)
        cout << v << " " << best_edge[v] << endl;
 
    for (auto e : g[v]) {
        int u = e.first, w = e.second;
        if (w < min_edge[u]) {
            min_edge[u] = w;
            best_edge[u] = v;
        }
    }
}

Можно не делать линейный поиск оптимальной вершины, а поддерживать его в приоритетной очереди.

Реализация за O(m \log n) — очень похожа на алгоритм Дейкстры:

In [ ]:
set< pair<int, int> > q;
int d[maxn];

while (q.size()) {
    v = q.begin()->second;
    q.erase(q.begin());
 
    for (auto e : g[v]) {
        int u = e.first, w = e.second;
        if (w < d[u]) {
            q.erase({d[u], u});
            d[u] = w;
            q.insert({d[u], u});
        }
    }
}

# Алгоритм Крускала

Заключается в том, чтобы добавлять рёбра в порядке возрастания их весов. Если ребро соединяет какие-то две уже соединенные вершины, то игнорировать. Работает по той же причине, что и Прим.

Чтобы реализовать этот алгоритм, нам сначала понадобится такая структура.

# Система непересекающихся множеств

Эта структура данных предоставляет следующие возможности. Изначально имеется несколько элементов, каждый из которых находится в отдельном (своём собственном) множестве. Структура поддерживает две операции:
* объединить два каких-либо множества
* запросить, в каком множестве сейчас находится указанный элемент

Обе операции будут выполняться в среднем почти за $O(1)$ (но не совсем — этот сложный вопрос будет разъяснен позже).

Множества элементов мы будем хранить в виде деревьев: одно дерево соответствует одному множеству. Корень дерева — это представитель (лидер) множества. Заведём массив `_p`, в котором для каждого элемента мы храним номер его предка в дерева. Для корней деревьев будем считать, что их предок — они сами.

Наивная реализация, которую мы потом ускорим:

In [ ]:
int _p[maxn];

int p (int v) {
    if (_p[v] == v)
        return v;
    else
        return p(_p[v]);
}

void unite (int a, int b) {
    a = p(a), b = p(b);
    _p[a] = b;
}

for (int i = 0; i < n; i++)
    _p[i] = i;

**Эвристика сжатия пути**. Оптимизируем работу функции `p`. Давайте перед тем, как вернуть ответ, запишем его в `_p` от текущей вершины, то есть переподвесим его за самую высокую.

Насколько лучше это сделает асимптотику? Выясняется, что $O(n \log n)$.

Тут должен быть мем из опросов.

**Ранговая эвристика**. Эта штука напрямую пытается минимизировать высоту дерева. Давайте делать переподвешивание за то, которое менее глубоко. Ну понятно, что тогда любое дерево будет не более логарифма.

**Весовая эвристика**. Давайте каждый раз подвешивать за более крупное. Работать будет быстро, так как .

Автор предпочитает именно её, потому что часто эти размеры компонент требуются сами по себе.

Оказывается, что сжатия + ранговая или сжатия + весовая работает быстро.

Асимптотика объединения обеих эвристик (сжатия путей и одной из ранговых) — O(a(n)), где a(n) — обратная функция Аккермана (очень медленно растущая функция, для всех адекватных чисел не превосходящая 4). Тратить время на изучения доказательства или даже чтения статьи на Википедии про функцию Аккермана автор не рекомендует.

In [ ]:
int _p[maxn], s[maxn];

int p (int v) { return (_p[v] == v) ? v : _p[v] = p(_p[v]); }

void unite (int a, int b) {
    a = p(a), b = p(b);
    if (s[a] > s[b]) swap(a, b);
    s[b] += s[a];
    _p[a] = b;
}

for (int i = 0; i < n; i++)
    _p[i] = i;

# Полезные свойства и классические задачи

* Если веса всех рёбер различны, то остов будет уникален.
* Минимальный остов является также и остовом с минимальным произведением весов рёбер (замените веса всех рёбер на их логарифмы)
* Минимальный остов является также и остовом с минимальным весом самого тяжелого ребра.
* Если вы решаете задачу, где ребра не добавляются, а удаляются, то можно попробовать решать задачу «с конца» и применить алгоритм Крускала.